In [16]:
import os

In [17]:
%pwd

'c:\\Users\\mrudu\\IKMLCourse\\IKProject\\MediWatch_project\\research'

In [ ]:
os.chdir("../")

In [13]:
%pwd

'c:\\Users\\mrudu\\IKMLCourse\\IKProject\\MediWatch_project\\research'

In [14]:
import pandas as pd

In [15]:
data = pd.read_csv("artifacts/data_ingestion/diabetic_data.csv")
data.head()

FileNotFoundError: [Errno 2] No such file or directory: 'artifacts/data_ingestion/diabetic_data.csv'

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101766 entries, 0 to 101765
Data columns (total 50 columns):
 #   Column                    Non-Null Count   Dtype 
---  ------                    --------------   ----- 
 0   encounter_id              101766 non-null  int64 
 1   patient_nbr               101766 non-null  int64 
 2   race                      101766 non-null  object
 3   gender                    101766 non-null  object
 4   age                       101766 non-null  object
 5   weight                    101766 non-null  object
 6   admission_type_id         101766 non-null  int64 
 7   discharge_disposition_id  101766 non-null  int64 
 8   admission_source_id       101766 non-null  int64 
 9   time_in_hospital          101766 non-null  int64 
 10  payer_code                101766 non-null  object
 11  medical_specialty         101766 non-null  object
 12  num_lab_procedures        101766 non-null  int64 
 13  num_procedures            101766 non-null  int64 
 14  num_

In [ ]:
data.isnull().sum()

encounter_id                    0
patient_nbr                     0
race                            0
gender                          0
age                             0
weight                          0
admission_type_id               0
discharge_disposition_id        0
admission_source_id             0
time_in_hospital                0
payer_code                      0
medical_specialty               0
num_lab_procedures              0
num_procedures                  0
num_medications                 0
number_outpatient               0
number_emergency                0
number_inpatient                0
diag_1                          0
diag_2                          0
diag_3                          0
number_diagnoses                0
max_glu_serum               96420
A1Cresult                   84748
metformin                       0
repaglinide                     0
nateglinide                     0
chlorpropamide                  0
glimepiride                     0
acetohexamide 

In [ ]:
data.shape

(101766, 50)

In [ ]:
data.columns

Index(['encounter_id', 'patient_nbr', 'race', 'gender', 'age', 'weight',
       'admission_type_id', 'discharge_disposition_id', 'admission_source_id',
       'time_in_hospital', 'payer_code', 'medical_specialty',
       'num_lab_procedures', 'num_procedures', 'num_medications',
       'number_outpatient', 'number_emergency', 'number_inpatient', 'diag_1',
       'diag_2', 'diag_3', 'number_diagnoses', 'max_glu_serum', 'A1Cresult',
       'metformin', 'repaglinide', 'nateglinide', 'chlorpropamide',
       'glimepiride', 'acetohexamide', 'glipizide', 'glyburide', 'tolbutamide',
       'pioglitazone', 'rosiglitazone', 'acarbose', 'miglitol', 'troglitazone',
       'tolazamide', 'examide', 'citoglipton', 'insulin',
       'glyburide-metformin', 'glipizide-metformin',
       'glimepiride-pioglitazone', 'metformin-rosiglitazone',
       'metformin-pioglitazone', 'change', 'diabetesMed', 'readmitted'],
      dtype='object')

In [ ]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataValidationConfig:
    root_dir: Path
    STATUS_FILE: str
    unzip_data_dir: Path
    all_schema: dict

In [ ]:
from Mediwatch_project.constants import *
from Mediwatch_project.utils.common import read_yaml, create_directories

In [ ]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_validation_config(self) -> DataValidationConfig:
        config = self.config.data_validation
        schema = self.schema.COLUMNS

        create_directories([config.root_dir])

        data_validation_config = DataValidationConfig(
            root_dir=config.root_dir,
            STATUS_FILE=config.STATUS_FILE,
            unzip_data_dir = config.unzip_data_dir,
            all_schema=schema,
        )

        return data_validation_config

In [ ]:
import os
import yaml
from Mediwatch_project import logger

In [ ]:
class DataValidation:
    def __init__(self, config: DataValidationConfig):
        self.config = config


    def validate_all_columns(self)-> bool:
        try:
            validation_status = None

            with open("schema.yaml", "r") as file:
                schema = Box(yaml.safe_load(file))

            # Access columns
            columns = schema.COLUMNS
            target_column = schema.target_column.name

            data = pd.read_csv(self.config.unzip_data_dir)

            def validate_schema(df, schema):
                missing_columns = [col for col in schema.COLUMNS if col not in df.columns]
                if missing_columns:
                    validation_status = False
                    with open(self.config.STATUS_FILE, 'w') as f:
                        f.write(f"Validation status: {validation_status}")
                    raise ValueError(f"Missing columns: {missing_columns}")

                for col, expected_type in schema.COLUMNS.items():
                    if col in df.columns:
                        if df[col].dtype.name != expected_type:
                            print(f"Warning: Column '{col}' has type {df[col].dtype}, expected {expected_type}")
                
                
                validation_status = True
                with open(self.config.STATUS_FILE, 'w') as f:
                    f.write(f"Validation status: {validation_status}")
                return validation_status

            validation_status = validate_schema(data, schema)

            return validation_status
        
        except Exception as e:
            raise e


In [ ]:
try:
    config = ConfigurationManager()
    data_validation_config = config.get_data_validation_config()
    data_validation = DataValidation(config=data_validation_config)
    data_validation.validate_all_columns()
except Exception as e:
    raise e

[2025-08-06 18:24:18,335: INFO: common: Raw YAML content from WindowsPath('config/config.yaml'): {'artifacts_root': 'artifacts', 'data_ingestion': {'root_dir': 'artifacts/data_ingestion', 'source_URL': 'https://raw.githubusercontent.com/dheeraj24797/MediWatch_project/main/diabetic_data.zip', 'local_data_file': 'artifacts/data_ingestion/data.zip', 'unzip_dir': 'artifacts/data_ingestion'}, 'data_validation': {'root_dir': 'artifacts/data_validation', 'unzip_data_dir': 'artifacts/data_ingestion/diabetic_data.csv', 'STATUS_FILE': 'artifacts/data_validation/status.txt'}} (type=<class 'dict'>)]
[2025-08-06 18:24:18,335: INFO: common: YAML file: config\config.yaml loaded successfully]
[2025-08-06 18:24:18,335: INFO: common: Raw YAML content from WindowsPath('params.yaml'): {'model_params': {'learning_rate': 0.01, 'n_estimators': 100}} (type=<class 'dict'>)]
[2025-08-06 18:24:18,339: INFO: common: YAML file: params.yaml loaded successfully]
[2025-08-06 18:24:18,343: INFO: common: Raw YAML conte